In [8]:
import json

from litellm import completion
from typing import Dict, List

from mdextractor import extract_md_blocks

OLLAMA_API_BASE = 'http://localhost:11434'
MODEL_NAME = 'gemini'

def generate_response(messages: List[Dict[str, str]]) -> str:
    response = {}
    if MODEL_NAME == 'gemini':
      response = completion(
          model='gemini/gemini-2.5-flash-lite',
          messages=messages,
          extra_body={
              'thinking_config': {
                  # Setting this to 0 should disable the reasoning process.
                  'thinking_budget': 0
              },
              'generation_config': {
                  'max_output_tokens': 2048,
                  'reasoning_effort': 'low'
              }
          }
      )
    elif MODEL_NAME == 'mistral':
      response = completion(
          model='ollama/mistral',
          api_base=OLLAMA_API_BASE,
          messages=messages,
      )

    print(response.get('usage', {}))

    return response['choices'][0]['message']['content']

def save_code_to_file(code: str, filename: str) -> None:
    try:
      with open(filename, 'w', encoding='utf-8') as f:
        f.write(code)
      print(f'Code saved to {filename}')
    except Exception as e:
      print(f'Error saving code to {filename}: {e}')

print("Done")

Done


### Basic interaction with the LLM

In [2]:
messages = [
    {'role': 'system', 'content': 'You are an expert software engineer that prefers functional programming.'},
    {'role': 'user', 'content': 'Write a function to swap the keys and values in a dictionary.'}
]

response = generate_response(messages)
print(response)

Usage(completion_tokens=328, prompt_tokens=41, total_tokens=369, completion_tokens_details=None, prompt_tokens_details=None)
 Sure! Here is a simple implementation of the function using Python, which is not strictly a functional programming language but it's widely used and suitable for this task. The function uses the built-in `dict()` constructor to create a new dictionary with swapped key-value pairs.

```python
def swap_keys_values(dictionary):
    return dict((value, key) for key, value in dictionary.items())

# Example usage:
my_dict = {'key1': 'value1', 'key2': 'value2'}
swapped_dict = swap_keys_values(my_dict)
print(swapped_dict)  # Output: {'value1': 'key1', 'value2': 'key2'}
```

If you want to implement this function using a more functional programming approach, consider the following solution in Python using list comprehensions and `itertools.zip_longest`:

```python
from itertools import zip_longest

def swap_keys_values(dictionary):
    return dict(zip_longest(*[iter(dict

You can pass almost anything to the LLM (including JSON docs

In [ ]:
code_spec = {
    'name': 'swap_keys_values',
    'description': 'Swaps the keys and values in a given dictionary.',
    'params': {
        'd': 'A dictionary with unique values.'
    },
}

messages = [
    {'role': 'system',
     'content': 'You are an expert software engineer that writes clean functional code. You always document your functions.'},
    {'role': 'user', 'content': f'Please implement: {json.dumps(code_spec)}'}
]

response = generate_response(messages)
print(response)

LLM don't have memory; you need to pass the conversation history to the LLM

In [ ]:
# 1st prompt
messages = [
    {'role': 'system', 'content': 'You are an expert software engineer that prefers functional programming.'},
    {'role': 'user', 'content': 'Write a function to swap the keys and values in a dictionary.'}
]

response = generate_response(messages)

# 2nd prompt
messages = [
    {'role': 'system', 'content': 'You are an expert software engineer that prefers functional programming.'},
    {'role': 'user', 'content': 'Write a function to swap the keys and values in a dictionary in Python.'},
    {'role': 'assistant', 'content': response},
    {'role': 'user', 'content': 'Please add documentation to the function.'}
]

response = generate_response(messages)
print(response)


In [ ]:
python_code = extract_md_blocks(response)
print(python_code[0])

### Practice
For this exercise, you should write a program that uses sequential prompts to generate any Python function based on user input. The program should:

First Prompt:

 - Ask the user what function they want to create
 - Ask the LLM to write a basic Python function based on the user’s description
 - Store the response for use in subsequent prompts
 - Parse the response to separate the code from the commentary by the LLM


Second Prompt:

 - Pass the code generated from the first prompt
 - Ask the LLM to add comprehensive documentation including:
 - Function description
 - Parameter descriptions
 - Return value description
 - Example usage
 - Edge cases


Third Prompt:

 - Pass the documented code generated from the second prompt
 - Ask the LLM to add test cases using Python’s unittest framework
 - Tests should cover:
 - Basic functionality
 - Edge cases
 - Error cases
 - Various input scenarios


In [9]:
# 1st prompt
messages = [
    {'role': 'system', 'content': 'You are an expert software engineer that is fluent in the Python programming language.'},
    {'role': 'user', 'content': 'Ask the user what function they want to create.'}
]

response = generate_response(messages)
print(response)

Usage(completion_tokens=74, prompt_tokens=26, total_tokens=100, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=None, text_tokens=26, image_tokens=None))
What kind of function would you like to create today? For example, are you looking to:

*   Perform a mathematical calculation?
*   Process some text?
*   Work with data (like lists or dictionaries)?
*   Interact with files?
*   Something else entirely?

Tell me a bit about what you want the function to do!


In [10]:
# 2nd prompt
messages += [
    {'role': 'assistant', 'content': response},
    {'role': 'user', 'content': 'I want a function that checks if a number is prime (call it is_prime), returns a boolean.'
                                'Think about the edge cases. Keep the explanations to a minimum. '
                                'I want the code in a markdown code block.'}
]

response = generate_response(messages)
print(response)

print('\n\n-------------------\n\n')

python_code = extract_md_blocks(response)
print(python_code[0])

Usage(completion_tokens=145, prompt_tokens=147, total_tokens=292, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=None, text_tokens=147, image_tokens=None))
```python
def is_prime(number: int) -> bool:
    """
    Checks if a given integer is a prime number.

    Args:
        number: The integer to check.

    Returns:
        True if the number is prime, False otherwise.
    """
    if number < 2:
        return False
    if number == 2:
        return True
    if number % 2 == 0:
        return False
    for i in range(3, int(number**0.5) + 1, 2):
        if number % i == 0:
            return False
    return True
```


-------------------


def is_prime(number: int) -> bool:
    """
    Checks if a given integer is a prime number.

    Args:
        number: The integer to check.

    Returns:
        True if the number is prime, False otherwise.
    """
    if number < 2:
        return False
    if number == 2:
   

In [11]:
# 3rd prompt
messages += [
    {'role': 'assistant', 'content': f'Code generated so far ```python\n{python_code[0]}\n```'},
    {'role': 'user', 'content': 'Please add documentation to the function.'
                                'I want parameters and return values documented.'
                                'I want example usage.'
                                'I want edge cases considered.'
                                'I want the code in a single markdown code block.'}
]

response = generate_response(messages)

python_code = extract_md_blocks(response)
print(python_code[0])

Usage(completion_tokens=386, prompt_tokens=335, total_tokens=721, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=None, text_tokens=335, image_tokens=None))
def is_prime(number: int) -> bool:
    """
    Checks if a given integer is a prime number.

    A prime number is a natural number greater than 1 that has no positive
    divisors other than 1 and itself.

    Edge Cases Considered:
    - Numbers less than 2 are not prime.
    - 2 is the only even prime number.
    - Negative numbers are not prime.

    Args:
        number: The integer to check for primality.

    Returns:
        True if the number is prime, False otherwise.

    Examples:
        >>> is_prime(2)
        True
        >>> is_prime(3)
        True
        >>> is_prime(4)
        False
        >>> is_prime(17)
        True
        >>> is_prime(1)
        False
        >>> is_prime(0)
        False
        >>> is_prime(-5)
        False
    """
    # 

In [12]:
messages += [
    {'role': 'assistant', 'content': f'Code generated so far ```python\n{python_code[0]}\n```'},
    {'role': 'user', 'content': 'Please add unit tests to the function unsing the unittest framework.'
                                'Keep explanations to a minimum.'
                                'Put everything in a single markdown code block.'}
]

response = generate_response(messages)

python_code = extract_md_blocks(response)
print(python_code[0])


Usage(completion_tokens=806, prompt_tokens=755, total_tokens=1561, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=None, text_tokens=755, image_tokens=None))
import unittest

def is_prime(number: int) -> bool:
    """
    Checks if a given integer is a prime number.

    A prime number is a natural number greater than 1 that has no positive
    divisors other than 1 and itself.

    Edge Cases Considered:
    - Numbers less than 2 are not prime.
    - 2 is the only even prime number.
    - Negative numbers are not prime.

    Args:
        number: The integer to check for primality.

    Returns:
        True if the number is prime, False otherwise.

    Examples:
        >>> is_prime(2)
        True
        >>> is_prime(3)
        True
        >>> is_prime(4)
        False
        >>> is_prime(17)
        True
        >>> is_prime(1)
        False
        >>> is_prime(0)
        False
        >>> is_prime(-5)
        Fa

In [13]:
save_code_to_file(python_code[0], './output/prime_checker.py')


Code saved to ./output/prime_checker.py
